In [1]:
# Imports a parser from cogent
from cogent.parse.fasta import MinimalFastaParser as parse

In [2]:
# applies for the whole segment
nprocs = 10

In [3]:
# Checking out data file.
# This file was created using the QC_basic notebook.
!head data/finalQC.fasta

>D3013.24_0
TTCATAATCAAAGTGTTTTTATGGCACTTTTAAAAAAATCCATATCCACCTTGTGTGCAATGTCATCTCACTGGAGGCCAGCTGGCTGTCAAAAGCCCGTTTGGTCACCTTTGGGATTTATATCTACTCAGAACTTTAGTGATTTTGTCTGAAAAATATTATGAATAACTTAATTCAAAATACAACTTTCAACAACGGATCTCTTGGCTCTCGC
>D1412.20_1
TTAACACTAATCCACACACTACTCAACCTAGCCTTTAGTTGCAGCCGAGGTGTTCGCCGTCAGGCAGCGCCGCAGCAGCAACCACAACAAACCTAATCTCAAAGGACTTTAACTAAGCCTTACCACAAAACCAAATTCTCAACGATGGATATCTTGGTTCCCAT
>D1412.18_2
TTACCGAGTTTACACCTCCCAAACCCCTGTGAACATACCTTAATGTTGCCTCGGCGGATCAGCCCGCGCCCCGTAAAACGGGACGGCCCGCCAGAGGACCCAAACTCTAATGTTTCTTATTGTAACTTCTGAGTAAAACAAACAAATAAATCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D3012.10_3
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTAAACCGTTGCTTCGGCGGGCGGCCCCGGGGTTCTCCCCGGGAGCCCCCGGGCCCCATCCCGGGCGCCCGCCGGAGTTCACCAAACTATTGATAATTTAGGGCCTCTCTGAGTCTTCTGTACCGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D713.15_4
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTAAACCGTTGCTTCGGCGGGCGGCCCCGGGGTTTACCCCCCGGGCGCCCCTGGGCCCCACCGCGGGCGCCCGCCGGAGGTCACCAAACTCTTGATAATTTATGGCCTCTCTGAGTCTTCTGTACT

In [4]:
%%bash
# Running the seq separation on mothur instead

mothur "#unique.seqs(fasta=data/finalQC.fasta)" > /dev/null

In [5]:
# Making a dictionary of the names files, splitting it into the first (ID) and second (commas list of all seqs in it)
# Then it counts their lengths and saves it in the dictionary

counts = {}

with open("data/finalQC.names") as f:
    for line in f:
        seedID, seqIDs = line.split("\t")
        count = len(seqIDs.split(","))
        counts[seedID] = count        

In [6]:
# Adds the counts from this dictionary to our finalQC.unique file so it looks like a usearch file with "size=XXX"

with open("data/finalQC.unique.usearch_names.fasta", "w") as f:
    for n, s in parse(open("data/finalQC.unique.fasta")):
        f.write(">%s;size=%s;\n%s\n"%(n,counts[n],s))  

In [7]:
!head data/finalQC.unique.usearch_names.fasta

>D3013.24_0;size=24757;
TTCATAATCAAAGTGTTTTTATGGCACTTTTAAAAAAATCCATATCCACCTTGTGTGCAATGTCATCTCACTGGAGGCCAGCTGGCTGTCAAAAGCCCGTTTGGTCACCTTTGGGATTTATATCTACTCAGAACTTTAGTGATTTTGTCTGAAAAATATTATGAATAACTTAATTCAAAATACAACTTTCAACAACGGATCTCTTGGCTCTCGC
>D1412.20_1;size=11899;
TTAACACTAATCCACACACTACTCAACCTAGCCTTTAGTTGCAGCCGAGGTGTTCGCCGTCAGGCAGCGCCGCAGCAGCAACCACAACAAACCTAATCTCAAAGGACTTTAACTAAGCCTTACCACAAAACCAAATTCTCAACGATGGATATCTTGGTTCCCAT
>D1412.18_2;size=19;
TTACCGAGTTTACACCTCCCAAACCCCTGTGAACATACCTTAATGTTGCCTCGGCGGATCAGCCCGCGCCCCGTAAAACGGGACGGCCCGCCAGAGGACCCAAACTCTAATGTTTCTTATTGTAACTTCTGAGTAAAACAAACAAATAAATCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D3012.10_3;size=1;
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTAAACCGTTGCTTCGGCGGGCGGCCCCGGGGTTCTCCCCGGGAGCCCCCGGGCCCCATCCCGGGCGCCCGCCGGAGTTCACCAAACTATTGATAATTTAGGGCCTCTCTGAGTCTTCTGTACCGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D713.15_4;size=33618;
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTAAACCGTTGCTTCGGCGGGCGGCCCCGGGGTTTACCCCCCGGGCGCCCCTGGGCCCCACCGCGGGCGCCCG

In [8]:
# Chuck looking to see how many Gb this file is.
!du -h data/finalQC.fasta

2.1G	data/finalQC.fasta


In [9]:
# Sequences are sorted by size
# Here the size of clusters - we are excluding the singletons here
# You would change minsize to 1 if you wanted to include singletons
# Or, you know, just not do this step.
# But you should just get rid of them.
!usearch -sortbysize data/finalQC.unique.usearch_names.fasta -output data/finalQC.unique_sorted.fasta -minsize 2

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:00 2.1Mb Reading data/finalQC.unique.usearch_names.fasta, 223Mb
00:00 225Mb 919410 (919.4k) seqs, min 50, avg 216, max 541nt
00:00 236Mb Getting sizes
00:01 243Mb Sorting 175798 sequences
00:02 244Mb  100.0% Writing data/finalQC.unique_sorted.fasta


In [10]:
# Checking data
# You can see here, the first two sequences we saw above are now gone.
!head data/finalQC.unique_sorted.fasta

>D3012.6_28;size=690611;
TTACAGAGTCGCAATACTCCGTAAACCATTGTGAACGTTACCTAACCGTTGCTTCGGCGGGCGGCGCCCGGGCCCTCGCG
GCCCGCGGCGCCCCCCGGCCCCTGCGGGCGCCCGCCGGAGGTAGACCAAACTCTTGAATTACATGGCCTCTCTGAGTCTT
CTGTACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D3013.8_50;size=590148;
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTTCAAACCGTTGCTTCGGCGGGCGGCCCGGGTCCGCCC
GGTGCCCCCTGGCCCCCTAGCGGGGCGCCCGCCGGAGGAAACCCAACTCTTGATTATTATGGCCTCTCTGAGTCTTCTGT
ACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D3013.6_37;size=320432;
TTACTGAGTACTACACTCTCTACCCTTTGTGAACTATTATACCTGTTGCTTCGGCGGCGCCCGCGAGGGTGCCCGCCGGT


In [11]:
# This is the clustering command.
# Default is 97% minimum ID.
# Not recommended to use more than 97%.
# Creates the centroids, or "seeds"
# Then you can take them out
!usearch -cluster_otus data/finalQC.unique_sorted.fasta -otus data/otus.fasta

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

02:59 331Mb  100.0% 38880 OTUs
                              
Input seqs  175798 (175.8k)
      OTUs  38880 (38.9k)
   Members  134029 (134.0k)
  Chimeras  2889
   Max mem  331Mb
      Time  02:59
Throughput  982.1 seqs/sec.



In [12]:
# Making another file
# Figure this out (what is this?)
# This is a script (fasta_number.py) that replaces fasta names with XXX1, XXX2, etc.
# In our case, it is replacing the names with OTU.1, OTU.2, etc., and outputs it into a file called otusn.fasta
!/opt/bioinfo/edgar_python_scripts/fasta_number.py data/otus.fasta OTU. > data/otusn.fasta

In [1]:
!head data/otusn.fasta

>OTU.1
TTACAGAGTCGCAATACTCCGTAAACCATTGTGAACGTTACCTAACCGTTGCTTCGGCGGGCGGCGCCCGGGCCCTCGCG
GCCCGCGGCGCCccccGGCCCCTGCGGGCGCCCGCCGGAGGTAGACCAAACTCTTGAATTACATGGCCTctctGAGTCTT
CTGTACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>OTU.2
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTTCAAACCGTTGCTTCGGCGGGCGGCCCGGGTCCGCCC
GGTGCCcccTGGCCcccTAGCGGGGCGCCCGCCGGAGGAAACCCAACTCTTGATTATTATGGCCTctctGAGTCTTCTGT
ACTGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>OTU.3
TTACTGAGTACTACACTctctACCCTTTGTGAACTATTATACCTGTTGCTTCGGCGGCGCCCGCGAGGGTGCCCGCCGGT


### Removing Chloroplast, Bacterial, and Archaeal sequences

In [2]:
# You do need to assign taxonomy in order to pull out the Euks., etc.
# I could cp these files from the server to wherever I need them.
# Input is your fasta file
# Output is a fasta with taxonomy assinged (still working with unique seqs)
# This didn't work first, because Chuck had to delete a "jobs" folder in the tmp directory.

!parallel_assign_taxonomy_uclust.py \
-r /var/seq_data/UNITE/31_01_16_s/sh_refs_qiime_ver7_97_s_31.01.2016.fasta \
-t /var/seq_data/UNITE/31_01_16_s/sh_taxonomy_qiime_ver7_97_s_31.01.2016.txt \
-O 10 \
-i data/otusn.fasta \
-o data/otusn_tax

In [3]:
# This makes a file of what we want to remove
# Could change this to pull out different groups.
# These primers actually had good Archaeal targets - so, it would be okay to include them.
!egrep "Chloroplast|Bacteria|Archaea|Unassigned|Mitochondria" \
data/otusn_tax/otusn_tax_assignments.txt \
| awk '{print $1}' > data/to_remove_tax.accnos

In [14]:
# wc is number of lines of the taxa that will be removed
!wc -l data/to_remove_tax.accnos

37314 data/to_remove_tax.accnos


In [5]:
# Looking at what you're removing
!head data/to_remove_tax.accnos

OTU.759
OTU.753
OTU.751
OTU.750
OTU.755
OTU.754
OTU.2323
OTU.2322
OTU.2321
OTU.2320


In [6]:
%%bash
# Remove.seqs command will actually remove these taxa
mothur "#remove.seqs(fasta=data/otusn.fasta, \
accnos=data/to_remove_tax.accnos)" #> /dev/null







mothur v.1.35.1
Last updated: 03/31/2015

by
Patrick D. Schloss

Department of Microbiology & Immunology
University of Michigan
pschloss@umich.edu
http://www.mothur.org

When using, please cite:
Schloss, P.D., et al., Introducing mothur: Open-source, platform-independent, community-supported software for describing and comparing microbial communities. Appl Environ Microbiol, 2009. 75(23):7537-41.

Distributed under the GNU General Public License

Type 'help()' for information on the commands that are available

Type 'quit()' to exit program



mothur > remove.seqs(fasta=data/otusn.fasta, accnos=data/to_remove_tax.accnos)
Removed 36980 sequences from your fasta file.

Output File Names: 
data/otusn.pick.fasta


mothur > quit()


In [2]:
! grep -c "OTU" data/otusn.pick.fasta

1900


### Mapping Reads

In [7]:
# Pulling out the sample identifier.
# It is adding a portion to the finalQC file that has the barcode label.
# Then we can use this later
# Now we will see how these reads map to the defined centroids (after removing EuK, etc.)
# Basically, we cut, cut, refined our fasta to make our OTU centroids.
# THEN, we went back to our original QC'd total fasta file and will throw it all against these nicely defined seeds.
# Anything that doesn't match, we won't keep.
!awk -F"_" \
'BEGIN{OFS=";"}{ if ( substr($1,0,1) == ">"){ print $0,"barcodelabel=",$1 } else { print $0 } }' \
data/finalQC.fasta | \
sed 's/;>//' > data/finalQC_usearchfmt.fasta

In [8]:
!head data/finalQC_usearchfmt.fasta

>D3013.24_0;barcodelabel=D3013.24
TTCATAATCAAAGTGTTTTTATGGCACTTTTAAAAAAATCCATATCCACCTTGTGTGCAATGTCATCTCACTGGAGGCCAGCTGGCTGTCAAAAGCCCGTTTGGTCACCTTTGGGATTTATATCTACTCAGAACTTTAGTGATTTTGTCTGAAAAATATTATGAATAACTTAATTCAAAATACAACTTTCAACAACGGATCTCTTGGCTCTCGC
>D1412.20_1;barcodelabel=D1412.20
TTAACACTAATCCACACACTACTCAACCTAGCCTTTAGTTGCAGCCGAGGTGTTCGCCGTCAGGCAGCGCCGCAGCAGCAACCACAACAAACCTAATCTCAAAGGACTTTAACTAAGCCTTACCACAAAACCAAATTCTCAACGATGGATATCTTGGTTCCCAT
>D1412.18_2;barcodelabel=D1412.18
TTACCGAGTTTACACCTCCCAAACCCCTGTGAACATACCTTAATGTTGCCTCGGCGGATCAGCCCGCGCCCCGTAAAACGGGACGGCCCGCCAGAGGACCCAAACTCTAATGTTTCTTATTGTAACTTCTGAGTAAAACAAACAAATAAATCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D3012.10_3;barcodelabel=D3012.10
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTAAACCGTTGCTTCGGCGGGCGGCCCCGGGGTTCTCCCCGGGAGCCCCCGGGCCCCATCCCGGGCGCCCGCCGGAGTTCACCAAACTATTGATAATTTAGGGCCTCTCTGAGTCTTCTGTACCGAATAAGTCAAAACTTTCAACAACGGATCTCTTGGTTCTGGC
>D713.15_4;barcodelabel=D713.15
TTACAGAGTTGCAAAACTCCCTAAACCATTGTGAACGTTACCTAAACCGTTGCTTCGG

In [9]:
# This is where the actual OTUs are being assigned. We choose 97% sequence ID threshold here.
# This might take a while - like 5 minutes
# Depending ont he clustering algorithm, like pairwise... it would take, like, days on the same number of processors.
# This is why usearch (centroid-based) is so much better
# But is it more biologically relevant? ... maybe, maybe not.
# Edgar is showing it's not that bad.

# We take our total QC data (modified above to have the sample ID extracted)
# We compare it to the otusn.pick.fasta database we made above
# We produce a readmap.uc file which tells us how the reads from our finalQC file map to the otusn seed database.

!usearch -usearch_global data/finalQC_usearchfmt.fasta \
-db data/otusn.pick.fasta \
-strand plus -id 0.97 \
-uc data/readmap.uc \
-threads 15

usearch v7.0.1090_i86linux32, 4.0Gb RAM (132Gb total), 40 cores
(C) Copyright 2013 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: chuck.peperanney@gmail.com

00:00  19Mb Reading data/otusn.pick.fasta, 418.0kb
00:00  19Mb 1900 seqs, min 50, avg 209, max 437nt
00:00  19Mb  100.0% Masking
00:00  20Mb  100.0% Word stats
00:00  22Mb  100.0% Building slots
00:00  22Mb  100.0% Build index
02:08 172Mb  100.0% Searching, 65.0% matched


In [10]:
# Makes an OTU table
# It will tell me the OTU ID, and then for all the samples, which OTUs it has sequences from.
!python /opt/bioinfo/edgar_python_scripts/uc2otutab.py data/readmap.uc > data/otu_table.txt

data/readmap.uc 100.0%   


In [11]:
# Issues with biom table formatting
!if [ -f data/otu_table.biom ]; then rm data/otu_table.biom; fi #This is to mitigate a biom bug
!biom convert -i data/otu_table.txt -o data/otu_table.biom --table-type "otu table"

In [12]:
# Issues with biom table formatting
!if [ -f data/otu_table_summary.txt ]; then rm data/otu_table_summary.txt; fi #This is to mitigate a biom bug
!biom summarize-table -i data/otu_table.biom -o data/otu_table_summary.txt

In [13]:
# This tells us the overall data info
# Num obs = OTUs
# total count = total seqs
# Chantal had 50% reduction after QC.

!cat data/otu_table_summary.txt

Num samples: 173
Num observations: 1900
Total count: 6476467
Table density (fraction of non-zero values): 0.221
Table md5 (unzipped): d4f9b15895cb062ce6db29ea4d76b602

Counts/sample summary:
 Min: 2183.0
 Max: 111388.0
 Median: 39314.000
 Mean: 37436.225
 Std. dev.: 20434.120
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
 D312.14: 2183.0
 D3013.14: 3591.0
 D1412.13: 3735.0
 D312.11: 4120.0
 D312.9: 4131.0
 D312.15: 4413.0
 D3013.15: 4528.0
 D1412.12: 4970.0
 D3013.13: 5129.0
 D1412.14: 5416.0
 D1413.15: 6712.0
 D312.10: 6907.0
 D713.17: 6908.0
 D312.12: 7058.0
 D312.13: 7341.0
 D712.16: 8097.0
 D713.16: 8704.0
 D312.23: 8749.0
 D3012.16: 8831.0
 D3013.16: 8933.0
 D312.24: 8952.0
 D1413.16: 9794.0
 D313.22: 11460.0
 D313.23: 11612.0
 D312.17: 12014.0
 D312.8: 12213.0
 D3013.12: 14053.0
 D3012.15: 14108.0
 D313.21: 14536.0
 D3012.13: 15318.0
 D3012.14: 15543.0
 D312.6: 15895.0
 D313.24: 17652.0
 D313.20: 17972.0
 D713.18